Import and initializing market data

In [1]:
from instruments.interestRateInstrument.interestRateSwap import InterestRateSwap
from instruments.interestRateInstrument.swaption import Swaption
from marketdata import init_marketdata



In [2]:
from marketdata.interestRateCurves import flat_ois_quote
from marketdata.swaptionVolatility import vols, parallelshift

Create a swaption

In [3]:
from utilities.Enums import Currency, SwapDirection, TradeDirection
fw_swap = InterestRateSwap(notional=1000000, currency=Currency.USD, timeToSwapStart_in_days=1 * 360,
                               timeToSwapEnd_in_days=3 * 360, swapDirection=SwapDirection.PAYER)
fair_rate = fw_swap.ql_swap.fairRate()
swaption = Swaption(underlyingSwap=fw_swap,
                    optionMaturity_in_days=1 * 360,
                    tradeDirection=TradeDirection.SHORT,
                    strikeFixedRate=fair_rate)
p0 = swaption.get_price()
print(p0)


1228.1266753088182


In [4]:
parallelshift(vols, 0.0001)
p1 = swaption.get_price()
DV01 = p1-p0
print(DV01)

2.0359650889952263


In [5]:
parallelshift(vols, -0.0001)
p2 = swaption.get_price()
p0==p2

True

Now let's look into the swaption delta which should be quite straight forward



In [6]:
p0 = swaption.get_price()

from marketdata.interestRateCurves import flat_ois_quote

flat_ois_quote.setValue(flat_ois_quote.value()+0.0001)

p1 = swaption.get_price()
DV01 = p1-p0
print(DV01)

0.0


In [7]:
flat_ois_quote.setValue(flat_ois_quote.value()-0.0001)
p2 = swaption.get_price()
p2==p0



True

let's try again by just evaluating the underlying swap

In [8]:
swap = swaption.underlying_swap

p0 = swap.get_price()
flat_ois_quote.setValue(flat_ois_quote.value()+0.0001)
p1 = swap.get_price()
print(p1-p0)
flat_ois_quote.setValue(flat_ois_quote.value()-0.0001)

1.6423768546810606


In [9]:
ps0 = swaption.underlying_swap.get_price()
po0 = swaption.get_price()

flat_ois_quote.setValue(flat_ois_quote.value()+0.0001)

ps1 = swaption.underlying_swap.get_price()
po1 = swaption.get_price()

Swap DV01

In [10]:
print(ps1-ps0)

1.6423768546810606


Swaption DV01

In [11]:
print(po1-po0)

0.0


In [12]:
swaption.ql_underlying_swap.fairRate()


0.02407922157050632